IMPORTANT! Before beginning any lab assignment, be sure to **make your own copy** of the notebook and name it "lastname - Lab 5" or something similar.

# Lab 5: Decision Trees in Python

In lecture, we have been using a custom built ID3 decision tree implementation to understand how decision trees work. In this lab, we will use the `scikit-learn` library to implement decision trees.  It is a bit different from the ID3 implementation we have been using, but it is a powerful and widely used library for machine learning in Python.

We are also going to explore the Evaluation Metrics used to assess the performance of machine learning models.

## Part A: Learning about Decision Tree Classifiers in `scikit-learn`

Decision trees in `scikit-learn` can be used for both classification and regression tasks. Some of the ideas discussed in lecture for ID3 decision trees, such as feature importance and tree visualization, are also applicable to `scikit-learn` decision trees.  However, they do NOT use the same algorithm or the same splitting criteria (ID3 uses Information Gain, while `scikit-learn` uses Gini Impurity for classification and Mean Squared Error for regression by default).

### Step 1: Get the Data we will use

Let's load a dataset about that most excellent of beverages, coffee!

In [ ]:
# Import needed packages and functions
import polars as pl
from pathlib import Path
import requests

# Set up Polars to show all columns when displaying DataFrames
pl.Config.set_tbl_formatting("ASCII_FULL_CONDENSED")
pl.Config.set_tbl_cols(-1)

# Set the URL and local filename
url = "https://raw.githubusercontent.com/JuanCab/csis446_lab05/refs/heads/main/data/coffee.csv"
local_filename = "coffee.csv"

# Check if the file already exists to avoid re-downloading
if not Path(local_filename).is_file():
    print(f"Downloading dataset from {url}...")
    # Download the dataset using requests library
    r = requests.get(url)
    # Check if the request was successful, if not, raise an error
    r.raise_for_status()

    # Save the content to a local file
    with open(local_filename, "wb") as f:
        f.write(r.content)
        print(f"Dataset downloaded and saved as '{local_filename}'.")

# Load the dataset from the local file into a Polars DataFrame
coffee_full_df = pl.read_csv(local_filename)

# Print number of features (columns) in the DataFrame
num_features = coffee_full_df.width
print(f"The dataset contains {num_features} features (columns).")

# This is too many features so we are going to only keep a subset
features_to_keep = ['species', 'aroma', 'acidity', 'body', 'flavor', 
                    'aftertaste', 'balance', 'uniformity', 'sweetness']
coffee_df = coffee_full_df.select(features_to_keep)

# Display the first few rows of the DataFrame to verify successful loading
coffee_df.head()

**Question**: Explore the dataset statistically by getting a summary of the dataset and checking for missing values.  How many unique species are in the dataset? **HINT**: Recall that in Polars, you can use the `.describe()` method to get a statistical summary of the dataframe, and the `.n_unique()` method to get the number of unique values in a column.

In [ ]:
# TODO: Get a statistical summary of the dataset

# TODO: Determine the number of unique species and then add comments 
# indicating the number of species.


### Step 2: Preparing the Data for `scikit-learn`

One of the major restrictions with `scikit-learn` decision tree classes is that they can only handle numerical features.  Categorical features must be converted to numerical features using feature transformation techniques before they can be used with `scikit-learn` decision trees.  Two common techniques are:

- **One hot encoding** creates a new binary feature for each category in the original feature. For example, if a feature has three categories (`A`, `B`, `C`), one-hot encoding would create three new features: `is_A`, `is_B`, and `is_C`. Each of these features would be 1 if the original feature was that category, and 0 otherwise.
- **Label encoding** assigns a unique integer to each category in the original feature. For example, if a feature has three categories (`A`, `B`, `C`), label encoding would assign 0 to `A`, 1 to `B`, and 2 to `C`. The original feature would then be replaced with these integer values.

Our target feature will be the `species` of coffee, which is a categorical feature with two possible values: `Arabica` and `Robusta`. **We will use label encoding** to convert this categorical feature into a numerical feature. This is done by creating a dictionary that maps each species to a unique integer. Then we use the `replace` method to create a new column with the encoded values. Since we only have two species, we can treat the label encoding as a binary feature where 1 indicates Robusta and 0 indicates Arabica.

In [ ]:
# Create a label encoding for the species. Since we only have two species,
# we can treat 'Robusta' as a binary variable where 1 indicates Robusta
# and 0 indicates Arabica. (We have to use "cast" here because replace
# returns the original data type by default)
species_mapping = {'Arabica': 0, 'Robusta': 1}
coffee_df = coffee_df.with_columns(
    pl.col('species').replace(species_mapping).cast(pl.Int64).alias('Robusta')
)

# Show the first few rows of the updated data
coffee_df.head()

**Question**: Explaing how you would need to modify the code above if there were more than two species in the dataset.

```
ENTER YOUR ANSWER HERE
```

### Step 3: Fitting the Model in a Way We Can Evaluate

For some unfortunate reason, your textbook chooses to address model evaluation *after* discussing all of the models.  However, in practical use, we need to know how well our models are performing in order to improve them and make informed decisions.

To assess a model's performance, we typically perform a **test/train split** of the dataset, where the data is split into a training set and a testing set.  The model is trained on the training set and then its performance is evaluated on the testing set.  This helps to ensure that the model is not overfitting to the training data and can generalize well to unseen data (*e.g.* the testing set).

Very commonly, we use a 70/30 or 80/20 split for the training and testing sets, respectively.  This means that 70% or 80% of the data is used for training, and the remaining 30% or 20% is used for testing. We can use the `train_test_split()` function from `sklearn.model_selection` to easily split the data into training and testing sets.  

The steps are:

1. Separate the features we will use in the fit (`X`) and the target variable (`y`).  **Remember you need to convert the Polars DataFrame to a Pandas DataFrame first using the `.to_pandas()` method as `scikit-learn` requires Pandas DataFrames.**
2. Use `X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=[some fraction], random_state=42)` to split the data into training and testing sets.  The `test_size` parameter specifies the proportion of the data to use for testing (here it is 20%), and the `random_state` parameter ensures that the split is reproducible (by fixing the random seed).
3. Train the model (in this case, a decision tree classifier) using the training data.
4. Use the trained model to make predictions on the testing data and compare those answers to the actual labels.

#### Modelling Step 1: Separate the Features and Target Variable

**Question**: Modify the code below to:

- `.select()` `uniformity` and `sweetness` as input features (`X`).  Remember to convert the result to a Pandas DataFrame using the `.to_pandas()` method.
- `.select()` the `Robusta` column as the target variable (`y`).  Remember to convert the result to a Pandas Series using the `.to_pandas()` method.

In [ ]:
# TODO: Select the input features using Polars syntax and convert to pandas
X = ...

# TODO: Select the target variable
y = ...

print(X)
print(y)

#### Modelling Step 2: Create Training and Testing Data

**Question**: Modify the code below to perform a 70/30 split of the data, producing `X_train`, `X_test`, `y_train`, and `y_test`.

In [ ]:
from sklearn.model_selection import train_test_split

# TODO: Create training and testing data with a test size of 30% and 
# random state of 123
X_train, X_test, y_train, y_test = train_test_split(...)

# Show the shapes of the resulting datasets to verify the split
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

**Question**: The cell below is going to produce a scatter plot of the training data where the points are colored based on their species.  Explain why this suggests a decision tree might work well for this dataset (**Hint**: Decision trees will split continuous features into binary splits.  A split of `uniformity` at 8.5 would mean all points with `uniformity` <= 8.5 go to one side of the tree ... how would this help in identifying the species?).


In [ ]:
import seaborn as sns

# Scatter plot of the input features colored by species of coffee
p1 = sns.scatterplot(data=X_train, 
                     x='uniformity', y='sweetness', 
                     hue=y_train['Robusta'], alpha=0.5)
p1.set(xlabel='Uniformity', ylabel='Sweetness')
p1.get_legend().set_title('Robusta?')

```
ENTER YOUR ANSWER HERE
```

#### Modelling Step 3: Train the Model

For this example, we want to use as a model a decision tree classifier to predict the species of coffee based on the `uniformity` and `sweetness` features.

Decision tree classifiers are implemented in `sklearn.tree` using [`DecisionTreeClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). They do not use the same algorithm as ID3,  but they do share some similar concepts.  Here are some important parameters and methods to be aware of:

- The `criterion` parameter specifies the function to measure the "information gain" of a split.  The default is `gini` for Gini Impurity, but you can also use `entropy` for Information Gain.
- The `max_depth` parameter specifies the maximum depth of the tree.  Limiting the depth can help prevent overfitting.
- The `.get_depth()` method can be used to get the depth of the tree.  This corresponds to how many questions are needed to identify the target label.

The following functions from `sklearn.tree` are also useful for visualizing and interpreting decision trees:

- `plot_tree()`: Plots the decision tree.
- `export_text()`: Exports the decision tree as text.

Now let's initialize and fit a decision tree classifier to the training data.  You'll notice the result is not quite as simple as the decision tree we saw earlier. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree

# Initialize the tree and fit the tree using the default parameters
coffee_clf = DecisionTreeClassifier()
coffee_clf.fit(X_train.values,y_train)

# Print the depth of the tree
print(f'The depth of the tree is {coffee_clf.get_depth()}')

# Show the tree as text. export_text() accepts the classifier model
# and the feature names (which are the column names of X_train)
print(export_text(coffee_clf, feature_names=X_train.columns))

The printed tree shows the splits made at each node, the number of samples at each node, the distribution of classes at each node, and the predicted class for each leaf node.  Let's visualize the tree using `plot_tree()`.  It uses the same arguments as `export_text()`, but produces a graphical representation of the tree.

In [ ]:
import matplotlib.pyplot as plt

# We will force the Decision Tree plot to be large so details can be seen
plt.figure(figsize=(20, 20))  # Width=20, Height=20 inches

# TODO: Plot the fitted tree. We set fontsize to keep the text readable and
# filled to color the boxes.  We assign to dummy variable _ to avoid text 
# output since this is the last line of the cell.

#### Modelling Step 4: Evaluate the Model using Predictions based on the Test Set

This is the part that your textbook has been delaying.  How do we know if our model is any good?  We can use several evaluation metrics to assess the performance of our decision tree classifier using the test data which was NOT used during training.

For a classifier, one of the obvious metrics is **accuracy**, which is the proportion of correct predictions made by the model Scikit-learn provides a convenient function called `accuracy_score()` in the `sklearn.metrics` module to calculate accuracy.  It takes two arguments: the true labels and the predicted labels. 

So let's see how accurate the decision tree classifier we trained is on the test data. We will use the `.predict()` method of the trained model to make predictions on the test data, `X_test`, saving the predicted labels as `y_predict`.  We know the actual labels (they are `y_test`) so we can use `accuracy_score()` to calculate the accuracy.

In [ ]:
from sklearn.metrics import accuracy_score

# Predict species for the test set.
# We need to use .values to convert from X_test DataFrame to an array
# which is what the .predict() method expects
y_pred = coffee_clf.predict(X_test.values)

# Print the accuracy
print('accuracy:', accuracy_score(y_test,y_pred))

**Question**: So how accurate is our decision tree classifier on the test data?  Is this good or bad?

```
ENTER YOUR ANSWER HERE.
```

 However, accuracy alone can be misleading, especially if the classes are imbalanced.  It can also be informative to look at what is referred to as the **confusion matrix**, which shows the number of true positives, true negatives, false positives, and false negatives. Scikit-learn provides a convenient function called `confusion_matrix()` in the `sklearn.metrics` module to compute the confusion matrix.  It takes two arguments: the true labels and the predicted labels. You can then visualize the confusion matrix using `ConfusionMatrixDisplay()`.  Let's do so below.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Plot the confusion matrix labelled with species names
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm, display_labels=['Arabica', 'Robusta']).plot()
plt.show()

**Question**: What is the confusion matrix for our decision tree classifier on the test data tell you about the model's performance?

```
ENTER YOUR ANSWER HERE.
```

**Question**: The confusion matrix does suggest a problem with using accuracy alone to evaluate the model. What is it?  **HINT**: How many Robusta coffees were in the test set compared to Arabica?  What if we just classified everything as Arabica?

```
ENTER YOUR ANSWER HERE
```

# Part B: Practicing Building a Decision Tree Classifier

Let's re-perform this analysis using the input features of `aroma` and `aftertaste` instead of `sweetness` and `uniformity`.  The next cell will go through all of the steps again, but using these new features.

## Step 1: Separate the Features and Target Variable

Separate the features `aroma` and `aftertaste` into our input features variable (`X2`) and continue to use the numerical `Robusta` feature as the target variable (`y2`).  Remember to convert the Polars DataFrame to a Pandas DataFrame first using the `.to_pandas()` method as `scikit-learn` requires Pandas DataFrames.

In [ ]:
# TODO: Select the input features using Polars syntax and convert to pandas
X2 = ...

# TODO: Select the target variable
y2 = ...

# Print the features and target variable to verify if they look reasonable
print(X2)
print(y2)

**Question**: Based on the plot above, do you think a decision tree will perform well on this dataset?  Why or why not?

```
ENTER YOUR ANSWER HERE
```

## Step 2: Create Training and Testing Data

Use an 80/20 split (instead of the 70/30 split we used previously) of the data, producing `X2_train`, `X2_test`, `y2_train`, and `y2_test`.

In [ ]:
from sklearn.model_selection import train_test_split

# TODO: Create training and testing data with a test size of 20% and 
# random state of 123
X2_train, X2_test, y2_train, y2_test = train_test_split(...)

# Show the shapes of the resulting datasets to verify the split
print(f'X2_train shape: {X2_train.shape}')
print(f'X2_test shape: {X2_test.shape}')
print(f'y2_train shape: {y2_train.shape}')
print(f'y2_test shape: {y2_test.shape}')

Let's plot this data up to see how well these features separate the two species.

In [ ]:
import seaborn as sns

# Scatter plot of the input features colored by species of coffee
p1 = sns.scatterplot(data=X2_train, 
                     x='aroma', y='aftertaste', 
                     hue=y2_train['Robusta'], alpha=0.5)
p1.set(xlabel='Aroma', ylabel='Aftertaste')
p1.get_legend().set_title('Robusta?')

**Question**: Based on the plot above, do you think a decision tree will perform well on this dataset?  Why or why not?

```
ENTER YOUR ANSWER HERE
```

## Step 3: Train the Model

Now train a new decision tree classifier using the new training data, call it `coffee_clf2`, and fit it to the training data.  Plot the tree using `plot_tree()` and print the depth of the tree using the `.get_depth()` method.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree

# TODO: Train the Model, then print the depth and plot the tree


**Question**: What is different about this tree compared to the previous one?  Why do you think this is?

```
ENTER YOUR ANSWER HERE.
```

## Step 4: Evaluate the Model using Predictions based on the Test Set

Determine the accuracy of this new decision tree classifier on the test data and display the confusion matrix.  Comment on the results in your code cell below.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# TODO: Evaluate the Model using Predictions based on the Test Set


# TODO: Comment on the results of using these two features for classification


## Part C: Using a Decision Tree Regressor

Decision trees can also be used for regression tasks, where the target variable is continuous rather than categorical. In `scikit-learn`, decision tree regression is implemented using the `DecisionTreeRegressor` class from the `sklearn.tree` module.

We can follow similar steps as we did for classification, but now we will be predicting a continuous variable.  This means the decision tree regressor, rather than trying to maximize information gain or Gini impurity at each split, will try to minimize the variance within each leaf node. This means we need to also change how we evaluate the results.  

The most common evaluation metric for regression tasks is the **Mean Squared Error (MSE)**, which measures the average squared difference between the predicted and actual values. A lower MSE indicates better model performance.


### Step 1: Load & Explore the Data

Load the California Housing dataset and inspect its basic properties.

In [ ]:
# Import required libraries
import polars as pl
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing

# Load the California Housing dataset built-in to scikit-learn
data = fetch_california_housing()

# Load all the input features into a Polars DataFrame
cali_housing_df = pl.DataFrame(data.data, schema=data.feature_names)

# Add the target feature (median house value in $100,000s) to the DataFrame
cali_housing_df = cali_housing_df.with_columns(pl.lit(data.target).alias("MedValue"))

# Get a list of all feature names except the target
feature_names = [name for name in cali_housing_df.columns if name != "MedValue"]
print(f'Feature names: {feature_names}')

# Show the first few rows of the dataset
cali_housing_df.head()

**Question**: What do you think the input features and target variable represent in this dataset?  Which might you expect to be most important for predicting the target variable?

```
ENTER YOUR ANSWER HERE.
```

**Question**: You can try to confirm your answer by checking the correlation coefficient between each input feature and the target feature of `MedValue`. Is your answer confirmed or not?

In [ ]:
# Let's run through all the input features (in feature_names) and compute
# their correlation coefficient (R^2) with the target `MedValue` feature.

# Use polars correlation coefficient method to compute these.
corr_matrix = cali_housing_df.corr()

# Pull the `MedValue` column of values (I can cut off the last column
# because I know it is `MedValue` given how I tacked it on at the end 
# of cali_housing_df above.  This is is not a general solution)
for i, feature in enumerate(corr_matrix.columns[:-1]):
    # Pull row i value from `MedValue` column
    R2 = corr_matrix["MedValue"][i]
    print(f"{feature:>10}: {R2:.5f}")

```
ENTER YOUR ANSWER HERE.
```

### Step 2: Split Data for Training & Testing

Your tasks:

- Define **features (X) and target (y)**.
- Split into **80% training, 20% testing** with a random state of 42.

In [ ]:
from sklearn.model_selection import train_test_split

# TODO: Define features (X) and target variable (y)
X = ...
y = ...

# TODO: Split the dataset into training (80%) and testing (20%) sets
# and set random_state to 42
X_train, X_test, y_train, y_test = train_test_split(...)

# Print dataset shapes
print(f"Training Set: {X_train.shape}, Testing Set: {X_test.shape}")

**NOTE**: We won't have you plot the data this time since it is higher dimensional, but you can still visualize the results later.  Keep in mind, this means you are going into this a little bit blindly, without knowing how well the features separate the target variable.

### Step 3: Train a Decision Tree Regressor

We'll train a **DecisionTreeRegressor** to predict housing prices. This is done similarly to the classifier, but using the regressor class, `DecisionTreeRegressor`.  The parameters are similar to those of the `DecisionTreeClassifier` except that the splitting criteria is not based on Information Gain but instead defaults to minimizing Mean Squared Error (MSE).

So build a regressor called `pricing_model` by training it on `X_train`, check its depth, and visualize the tree using `plot_tree()`.

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text

# TODO: Initialize a Decision Tree Regressor to predict housing prices
pricing_model = ...

# TODO: Train the model on the training data

# TODO: Print tree depth

# TODO: Print the tree as text, but limit to max depth of 3 for readability
# Hint: use the "max_depth" parameter of export_text

# Create a larger figure for better visibility
plt.figure(figsize=(20, 12))  # Width=20, Height=12 inches

# TODO: Plot the tree but limit the depth to 3 for better visualization
# Hint: use the "max_depth" parameter of plot_tree

# Add title (assign to dummy variable to avoid text output)
_ = plt.title("Decision Tree Visualization (Limited to Depth 3)", fontsize=16)

**Question**: What are the first three splits in the tree (covering the first 2 levels)?  What do they indicate about the importance of the features?

```
ENTER YOUR ANSWER HERE
```

**Question:** What does tree depth tell us about the model?

```
ENTER YOUR ANSWER HERE
```

### Step 4: Evaluate Model Performance

With a classifier, we used accuracy and the confusion matrix to evaluate performance. Since this is a **regression task**, we'll use **Root Mean Squared Error (RMSE)** of the output continuous variable as an evaluation metric and our goal will be to minimize it (instead of maximizing accuracy).  Recall what this does is measure the average squared difference between the predicted and actual values (which makes the error positive) and then takes the square root to return to the original units.

Let's use the trained model to make predictions on the test data and calculate the MSE.  Scikit-learn provides a convenient function called `root_mean_squared_error()` in the `sklearn.metrics` module to calculate MSE.  It takes two arguments: the true values and the predicted values.

In [ ]:
from sklearn.metrics import root_mean_squared_error

# TODO: Use the trained model to make predictions on the test set
y_pred = pricing_model.predict(X_test.values)

# TODO: Compute Root Mean Squared Error (RMSE) which compares y_test and 
# y_pred
rmse = root_mean_squared_error(...)
print(f"Root Mean Squared Error: {rmse:.10f}")

# TODO: Remember the unit of the target variable is $100,000s, print
# the RMSE in dollars
print(f"RMSE in Dollars: ${...}")

**Question:**:** Is the error acceptable for a real-world housing price prediction?

```
ENTER YOUR ANSWER HERE
```

### Step 5: Addressing Overfitting by Experimenting with `max_depth`

With 21 levels, our decision tree regressor is likely overfitting the training data, essentially memorizing it rather than learning general patterns. One way to address overfitting is to limit the depth of the tree using the `max_depth` parameter when initializing the `DecisionTreeRegressor`. By restricting the depth, we can prevent the model from becoming too complex and help it generalize better to unseen data.

This limiting of the tree depth is a form of **regularization**, which is a technique used to reduce overfitting by adding constraints to the model. Since we are limiting the breadth of the tree, this is typically called **pruning** in decision tree terminology. We will explore how changing the `max_depth` parameter affects the model's performance, but be aware there are **pruning techniques** that can be applied after the tree is built that prune specific branches of the tree rather than limiting the overall depth.

Let's mess around a bit with these parameters and check out what it does. Your task here is to train decision trees and:

- Limit the **depth of the tree** to prevent overfitting using the `max_depth` parameter of `DecisionTreeRegressor`.
- Compare **RMSE at different depths**.

**Question**: Get the following cell working and then explain in the comments what the resulting plot is showing you.

In [ ]:
# TODO: Explain what depths are being tested here (Replace this comment
# with your explanation)
depths = range(1, pricing_model.get_depth() + 1)

# Make lists to save depths and corresponding RMSEs
depths_list = []
rmse_list = []

for depth in depths:
    # TODO: Initialize a Decision Tree with max_depth = depth
    pruned_pricing_model = DecisionTreeRegressor(max_depth=...)

    # TODO: Train the model
    pruned_pricing_model.fit(...)

    # TODO: Make predictions and compute RMSE
    y_pred = pruned_pricing_model.predict(...)
    rmse = root_mean_squared_error(y_test, y_pred)

    # Save depth and RMSE to lists
    depths_list.append(depth)
    rmse_list.append(rmse)

# Plot depth vs RMSE
plt.figure(figsize=(10, 6))
plt.plot(depths_list, rmse_list, marker='o')
plt.title('Decision Tree Depth vs RMSE', fontsize=16)
plt.xlabel('Tree Depth', fontsize=14)
plt.ylabel('Root Mean Squared Error (RMSE)', fontsize=14)
plt.xticks(depths_list)
plt.grid()
plt.show()

# TODO: Once you have the code working, explain what the plot is showing you
# below (in place of this comment).

**Question**: What is the optimal tree depth based on the plot?  Why do you think this tree is better than the full depth tree we built earlier?


```
ENTER YOUR ANSWER HERE.
```

## What you need to submit

Once you have completed this lab, you need to submit your work. You should submit the `.ipynb` notebook file. To do this, go to the File menu and select **Download > Download .ipynb** (this is the native Jupyter notebook format).  Submit that file to the Lab 5 dropbox on D2L.